<a href="https://colab.research.google.com/github/RaoSunny124/AI_201/blob/main/Assignment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import userdata
import os
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [25]:
!pip install --upgrade --quiet google-genai


In [26]:
!pip install -q gtts

In [27]:
from IPython.display import display, Audio ,Markdown

In [28]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"


In [30]:

from google.colab import files

uploaded = files.upload()
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')


Saving filename.mp4 to filename (2).mp4
Uploaded file: filename (2).mp4


In [31]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path="filename.mp4")
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

pottery_video = upload_video('filename.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/wv2dpvsx4e7y


In [32]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)
Scenes = response.text
Markdown(Scenes)


```json
[
  {
    "timecode": "00:00",
    "caption": "A man with short, dark hair is speaking to the camera in what looks like a living room. The man has a beard and mustache and is wearing a striped v-neck shirt. Behind him, there is a wall with a decorative wall hanging and some clothing draped on top of a table, which is covered with a floral tablecloth. There is a fan to the left and a wooden dresser to the right. A small blue pillow is visible in the lower part of the shot, to the left."
  },
  {
    "timecode": "00:02",
    "caption": "The man continues speaking, providing his name, location, age and hobbies. The camera is steady, so the focus remains on his upper body."
  },
  {
    "timecode": "00:18",
    "caption": "The man continues to speak about his hobbies, family, and living situation. The camera remains steady."
  }
]
```

In [33]:
from gtts import gTTS
from google.genai.types import Content, Part


# Your prompt
prompt = """For each scene in this video,
            generate captions that describe the audio along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

# Generate response
response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
            ]),
        prompt,
    ]
)

# Get the text response
output_text = response.text

# Print and speak the output
print(output_text)

```json
[
  {
    "timecode": "00:00",
    "caption": "A man is speaking in front of the camera. He begins by saying \"Assalam-o-Alaikum\"."
  },
  {
    "timecode": "00:01",
   "caption": "The man continues to speak, saying \"My name is Hasan. I am from Pakistan.\""
  },
    {
    "timecode": "00:05",
   "caption": "The man continues to speak, saying \"My age is 19. I live in Bihari, Punjab, Pakistan.\""
  },
  {
    "timecode": "00:10",
   "caption": "The man is still talking, saying \"I am studying Python from PIAIC in batch 62.\""
  },
  {
    "timecode": "00:17",
   "caption": "The man continues to speak, saying \"My hobbies is watching movies and vlogs also.\""
  },
  {
   "timecode": "00:24",
  "caption": "The man continues to speak, saying \"Okay, I have one younger sister.\""
  },
    {
    "timecode": "00:30",
    "caption": "The man continues talking, saying \"My father works as a farmer and my mother is a housewife.\""
  }
]
```


In [34]:
from google.genai.chats import Chat

# Initiate a chat
chat: Chat = client.chats.create(
    model=model
)

# Send chat a message
response = chat.send_message(output_text)
Markdown(response.text)


```json
[
  {
    "timecode": "00:00",
    "speaker": "Hasan",
    "dialogue": "Assalam-o-Alaikum"
  },
  {
    "timecode": "00:01",
   "speaker": "Hasan",
    "dialogue": "My name is Hasan. I am from Pakistan."
  },
    {
    "timecode": "00:05",
   "speaker": "Hasan",
    "dialogue": "My age is 19. I live in Bihari, Punjab, Pakistan."
  },
  {
    "timecode": "00:10",
   "speaker": "Hasan",
    "dialogue": "I am studying Python from PIAIC in batch 62."
  },
  {
    "timecode": "00:17",
   "speaker": "Hasan",
    "dialogue": "My hobbies is watching movies and vlogs also."
  },
  {
   "timecode": "00:24",
   "speaker": "Hasan",
    "dialogue": "Okay, I have one younger sister."
  },
    {
    "timecode": "00:30",
    "speaker": "Hasan",
    "dialogue": "My father works as a farmer and my mother is a housewife."
  }
]
```


In [37]:
response = chat.send_message("can u tell me my name find in this vedio?")
Markdown(response.text)


Yes, your name in the video is **Hasan**.


In [39]:
response = chat.send_message("What is my person age and country in the video?")
Markdown(response.text)


Based on the provided JSON, your age is **19** and your country is **Pakistan**.
